# 十分钟强化学习第四讲：蒙特卡罗方法
在Policy Iteration中我们知道状态转移函数的所有细节，  
但实际情况下，我们只能进行与环境的交互，不知道状态转移的具体细节  
蒙特卡罗方法实际上就是不断模拟来获得一个统计值来近似真实值

![Alt text](frozen_lake.gif)

In [1]:
from help import FrozenLake, print_policy, test_game
import numpy as np

In [2]:
def select_action(state, Q, mode="both"):
    if mode == "explore":
        return np.random.randint(len(Q[state]))
    if mode == "exploit":
        return np.argmax(Q[state])
    if mode == "both":
        if np.random.random() > 0.5:
            return np.argmax(Q[state])
        else:
            return np.random.randint(len(Q[state]))

In [3]:
def play_game(env, Q ,max_steps=200):
    state = env.reset()
    episode = []
    finished = False
    step = 0

    while not finished:
        action = select_action(state, Q, mode='both')
        next_state, reward, finished = env.step(action)
        experience = (state, action, finished,reward)
        episode.append(experience)
        if step >= max_steps:
            break
        state = next_state
        step += 1

    return np.array(episode,dtype=object)

如何算Q：计算单条轨迹下的平均return

In [4]:
def monte_carlo(env, episodes=10000, test_policy_freq=1000):
    nS, nA = 16, 4
    Q = np.zeros((nS, nA), dtype=np.float64)
    returns = {} 

    for i in range(episodes): 
        episode = play_game(env, Q)
        visited = np.zeros((nS, nA), dtype=bool)

        for t, (state, action, _, _) in enumerate(episode):
            state_action = (state, action)
            # 状态行为对只取第一次出现的
            if not visited[state][action]:
                visited[state][action] = True
                discount = np.array([0.9**i for i in range(len(episode[t:]))])
                reward = episode[t:, -1]
                G = np.sum( discount * reward)
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]  

                Q[state][action] = sum(returns[state_action]) / len(returns[state_action])
                #Q[state][action] = Q[state][action] + 1/len(returns[state_action]) * (G - Q[state][action])
        pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]

        if i % test_policy_freq == 0:
                print("Test episode {} Reaches goal {:.2f}%. ".format
                (i, test_game(env, pi)*100))
            
    return pi,Q

**增量平均的数学推导**

1. **样本均值的基本公式**

   给定一个数据序列：$x_1, x_2, \dots, x_n$，其算术平均值为：

   $$
   \bar{x}_n = \frac{1}{n} \sum_{i=1}^{n} x_i
   $$

2. **增量平均的递推公式推导**

   当新数据点 $x_{n+1}$ 到来时，新的均值为：

   $$
   \bar{x}_{n+1} = \frac{1}{n+1} \sum_{i=1}^{n+1} x_i
   $$

   将求和部分拆分：

   $$
   \bar{x}_{n+1} = \frac{1}{n+1} \left( \sum_{i=1}^{n} x_i + x_{n+1} \right)
   $$

   由于 $\sum_{i=1}^{n} x_i = n \cdot \bar{x}_n$，代入后得到：

   $$
   \bar{x}_{n+1} = \frac{1}{n+1} \left( n \cdot \bar{x}_n + x_{n+1} \right)
   $$

   展开并整理：

   $$
   \bar{x}_{n+1} = \frac{n}{n+1} \bar{x}_n + \frac{1}{n+1} x_{n+1}
   $$

   进一步表示为增量形式：

   $$
   \bar{x}_{n+1} = \bar{x}_n + \frac{1}{n+1} \left( x_{n+1} - \bar{x}_n \right)
   $$

   其中，$\frac{1}{n+1}$ 是学习率，随着样本数量的增加而减小。

In [5]:
env = FrozenLake()

In [6]:
policy_mc,Q = monte_carlo(env,episodes=20000)

Test episode 0 Reaches goal 0.00%. 
Test episode 1000 Reaches goal 40.00%. 
Test episode 2000 Reaches goal 32.00%. 
Test episode 3000 Reaches goal 49.00%. 
Test episode 4000 Reaches goal 37.00%. 
Test episode 5000 Reaches goal 38.00%. 
Test episode 6000 Reaches goal 53.00%. 
Test episode 7000 Reaches goal 46.00%. 
Test episode 8000 Reaches goal 42.00%. 
Test episode 9000 Reaches goal 37.00%. 
Test episode 10000 Reaches goal 48.00%. 
Test episode 11000 Reaches goal 56.00%. 
Test episode 12000 Reaches goal 61.00%. 
Test episode 13000 Reaches goal 49.00%. 
Test episode 14000 Reaches goal 53.00%. 
Test episode 15000 Reaches goal 47.00%. 
Test episode 16000 Reaches goal 43.00%. 
Test episode 17000 Reaches goal 44.00%. 
Test episode 18000 Reaches goal 39.00%. 
Test episode 19000 Reaches goal 46.00%. 


In [7]:
print_policy(policy_mc)
print('Reaches goal {:.2f}%. '.format(test_game(env, policy_mc)*100))

Policy:
| 00      v | 01      ^ | 02      < | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 49.00%. 


蒙特卡罗方法的缺点：
- 要等到游戏一轮完结后才更新
- 利用的信息中噪声较多，学习效率较低